






### Instruction
Spam filtering is a beginner’s example of the document classification task which involves classifying an email as spam or non-spam (a.k.a. ham) mail. An email dataset will be provided. We will use the following steps to build this application:
1) Preparing the text data
2) Creating a word dictionary
3) Feature extraction
4) Training the classifier
5) Checking the results on the test set

### Preparing the text data
The data-set used here, is split into a training set and a test set containing 702 mails and 260 mails respectively, divided equally between spam and ham mails. You will easily recognize spam mails as it contains `spmsg` in its filename.

In any text mining problem, text cleaning is the first step where we remove those words from the document which may not contribute to the information we want to extract. Emails may contain a lot of undesirable characters like punctuation marks, stop words, digits, etc which may not be helpful in detecting the spam email. The emails in Ling-spam corpus have been already preprocessed in the following ways:

1. **Removal of stop words** – Stop words like “and”, “the”, “of”, etc are very common in all English sentences and are not very meaningful in deciding spam or legitimate status, so these words have been removed from the emails.

2. **Lemmatization** – It is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. For example, “include”, “includes,” and “included” would all be represented as “include”. The context of the sentence is also preserved in lemmatization as opposed to stemming (another buzz word in text mining which does not consider meaning of the sentence)

We still need to remove the non-words like punctuation marks or special characters from the mail documents. There are several ways to do it. Here, we will remove such words after creating a dictionary, which is a very convenient method to do so since when you have a dictionary; you need to remove every such word only once.

### Creating word dictionary
We will only perform text analytics on the content to detect the spam mails. As the first step, we need to create a dictionary of words and their frequency. For this task, a training set of 700 mails is utilized. This python function will create the dictionary for you.
```Python
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    # Paste code for non-word removal here
    
    return dictionary
```

Once the dictionary is created we can add just a few lines of code written below to the above function to remove non-words. Absurd single characters in the dictionary which are irrelevant here are also removed. Do not forget to insert the below code in the function of make_Dictionary:
```python
list_to_remove = list(dictionary.keys())
for item in list_to_remove:
    if item.isalpha() == False:
        del dictionary[item]
    elif len(item) == 1:
        del dictionary[item]
dictionary = dictionary.most_common(3000)
```

In [3]:
import os
import numpy as np
from collections import Counter
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir (train_dir)]
    all_words = []
    for mail in emails:
        with open (mail) as m:
            for i,line in enumerate (m) :
                if i == 2:
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    # Paste code for non-word removal here
    return dictionary

The dictionary can be seen by the command “print dictionary”. You may find some absurd word counts to be high but don’t worry, it’s just a dictionary and you always have a chance to improve it later. If you use the provided dataset, make sure your dictionary has some of the entries given below as most frequent words. Here 3000 most frequently used words are chosen in the dictionary.

In [4]:
# To show the most frequent words in train-mails
dictionary = make_Dictionary('ling-spam/train-mails')

### Feature Extraction Process
Once the dictionary is ready, we can extract word count vector (our feature here) of 3000 dimensions for each email of the training set. Each **word count vector** contains the frequency of 3000 words in the training file. Of course you might have guessed by now that most of them will be zero. Let us take an example. Suppose we have 500 words in our dictionary. Each word count vector contains the frequency of 500 dictionary words in the training file. Suppose the text in the training file is “Get the work done, work done”, then it will be encoded as $$[0,0,0,0,0,…….0,0,2,0,0,0,……,0,0,1,0,0,…0,0,1,0,0,……2,0,0,0,0,0]$$ Here, all the word counts are placed at the 296th, 359th, 415th, 495th elements of the word count vector in the length of 500 and the rest are zero.

The below python code will generate a feature vector matrix whose rows denote 700 files of the training set and columns denote 3000 words of the dictionary. The value at index ${ij}$ will be the number of occurrences of the $j^{th}$ word of the dictionary in the $i^{th}$ file

In [5]:
def extract_features_and_label(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    label_matrix = np.zeros((len(files),1))
    docID = 0
    _i = 0
    print(len(files))
    for fil in files:
        if "spmsg" in fil:
            label_matrix[docID, 0] = 1
        _i+=1
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID]+=1
            docID = docID + 1
        print('\r','done {} files'.format(_i),flush=True,end='')
    return features_matrix, label_matrix

In [6]:
X_train, y_train = extract_features_and_label("ling-spam/train-mails")

702
 done 702 files

In [7]:
X_test, y_test = extract_features_and_label("ling-spam/test-mails")

260
 done 1 files

 done 260 files

### Training the Classifiers
Here you should write your Naïve Bayes classifiers after fully understanding its principle.

In [18]:
########### Write Your Code Here ###########
class NaiveBayes():

    def fit(self, X, y):
        y = y.astype(int)
        self.X = X
        self.y = y
        self.classes = np.unique(y)
        self.parameters = {}
        for i, c in enumerate(self.classes):
            # Calculate the mean, variance, prior probability of each class
            index = np.where(y==c)[0]
            X_Index_c = X[index] 
            X_index_c_mean = np.mean(X_Index_c, axis=0, keepdims=True)
            X_index_c_var = np.var(X_Index_c, axis=0, keepdims=True)
            X_index_c_mean[np.where(X_index_c_mean==0)] += 1e-10
            X_index_c_var[np.where(X_index_c_var==0)] += 1e-10
            parameters = {"mean": X_index_c_mean, "var": X_index_c_var, "prior": X_Index_c.shape[0] / X.shape[0]}
            self.parameters["class" + str(c)] = parameters
    
    def predict(self, X):
            # return class with highest probability
            # Complete code for naive Bayes algorithm
            def logGaussP(mean, var, x):
                return -np.square(x-mean) / (2 * var) - np.log(np.sqrt(2*np.pi*var))

            prediction = np.zeros((X.shape[0]))
            for j, text in enumerate(X):
                prob = np.zeros(self.classes.shape)
                for i, c in enumerate(self.classes):   
                    m = self.parameters["class" + str(c)]["mean"][0]
                    v = self.parameters["class" + str(c)]["var"][0]
                    p = self.parameters["class" + str(c)]["prior"]
                    prob[i] = np.sum(logGaussP(m, v, text)) + np.log(p)   # calculate cumulative product of gaussian probabilities
                prediction[j] = self.classes[np.argmax(prob)]

            return prediction
    '''
    
    def predict(self, X):
        # return class with highest probability
        predictions = []
        
        # Complete code for naive Bayes algorithm
        for x in X:
            posteriors = []

            # Calculate posterior probability for each class
            for c in self.classes:
                parameters = self.parameters["class" + str(c)]
                prior = parameters["prior"]
                mean = parameters["mean"]
                var = parameters["var"]
                # print("bad mean:", mean[mean<=0])
                # print("bad var:", var[var<=0])

                # Calculate the Gaussian likelihood
                likelihood = np.prod(1.0 / (np.sqrt(2 * np.pi * var)) * np.exp(-(x - mean) ** 2 / (2 * var)), axis=1)
                # print("likelihood =", likelihood)

                # Calculate the posterior probability
                posterior = prior * likelihood
                posteriors.append(posterior)

            # Choose the class with the highest posterior probability as the prediction
            prediction = self.classes[np.argmax(posteriors)]
            predictions.append(prediction)
            # print("pridiction is", predictions)
            # break

        return np.array(predictions)
    '''


In [19]:
from sklearn.metrics import confusion_matrix

#Call the Naive Bayes algorithm, which we wrote ourselves
model = NaiveBayes()
model.fit(X_train,y_train) 
result = model.predict(X_test)
print (confusion_matrix(y_test, result))

[[129   1]
 [  9 121]]


### Checking Performance
The test set contains 130 spam emails and 130 non-spam emails. Please compute accuracy, recall, F-1 score to evaluate the performance of your spam filter.

In [35]:
########### Write Your Code Here ###########
from sklearn.metrics import accuracy_score, recall_score, f1_score
print(accuracy_score(y_test, result))
print(recall_score(y_test, result))
print(f1_score(y_test, result))
############################################

0.9615384615384616
0.9307692307692308
0.9603174603174605


### Questions
1. Describe another real-world application where the naïve Bayes method can be applied
2. What are the strengths of the naïve Bayes method; when does it perform well?
3. What are the weaknesses of the naïve Bayes method; when does it perform poorly?
4. What makes the naïve Bayes method a good candidate for the classification problem, if you have enough knowledge about the data?

In [2]:
import numpy as np
# 创建一个二维数组
arr = np.array([[1, 2, 3], [4, 5, 6]])

# 计算每行的乘积
row_products = np.prod(arr, axis=1)

# 计算每列的乘积
col_products = np.prod(arr, axis=0)

# 打印结果
print("Row Products:", row_products)
print("Column Products:", col_products)

Row Products: [  6 120]
Column Products: [ 4 10 18]
